In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import mixture, decomposition, manifold, cluster, metrics

from collections import defaultdict

In [ ]:
dfs = []
for i in range(2006,2020):
    dfs.append(np.load('tables_{}.pkl'.format(i),allow_pickle=True))

In [ ]:
pv = defaultdict(list)
poses = defaultdict(list)
vorps = defaultdict(list)
initial_list = set()
for idx,df in enumerate(dfs):
    for team in df.values():
        for row in team['advanced'].itertuples(): #advanced
            if row[3] > 400:#row[2]*row[4] > 400:
                v = row[6:-10] #advanced
                #v = row[5:-3] #per_poss
                pv[row[0]].append(np.array([_ if _!='' else 0 for _ in v])) #-6 # -11
                poses[row[0]].append(team['roster'].loc[row[0]]['Pos'])
                vorps[row[0]].append(row[-1])


In [ ]:
row

In [ ]:
pva = {k: np.array(v).mean(0) for k,v in pv.items() }

In [ ]:
t = np.stack(pva.values())
#t = np.stack(sum(pv.values(),[]))
#t = np.nan_to_num(t)

#t = t[:,[0,4,5,9]]

t = (t-t.mean(0))/t.std(0)
print(t.shape)

In [ ]:
import umap


In [ ]:
pca_fit = decomposition.PCA()
res = pca_fit.fit_transform(t)
res = umap.UMAP().fit_transform(t)
#res = manifold.TSNE(perplexity=15).fit_transform(t)
print(pca_fit.components_[:2])

In [ ]:
team['advanced'].columns[5:-10]

In [ ]:
plt.scatter(res[:,0],res[:,1],s=4)

In [ ]:
for n_clusters in range(2,5):
    clusterer = cluster.KMeans(n_clusters=n_clusters,n_init=10 )
    cluster_labels = clusterer.fit_predict(t)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = metrics.silhouette_score(t, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

In [ ]:
clusterer = cluster.KMeans(n_clusters=3,n_init=10 )
cluster_labels = clusterer.fit_predict(t)
pose_short = np.array([max(set(lst), key=lst.count) for _,lst in poses.items()])

In [ ]:
plt.figure(dpi=400)
for i in np.unique(cluster_labels):
    plt.scatter(res[np.where(cluster_labels==i),0],res[np.where(cluster_labels==i),1],s=4)
    for i,name in enumerate(pva.keys()):
        if np.array(vorps[name]).mean() > 2:
            print(name,res[i])
            plt.text(res[i,0],res[i,1],name.split()[1],ha='center',va='center',size=8)
#plt.xticks([],[])
#plt.yticks([],[])
plt.title('NBA Positions based on per-possession statistics (via UMAP)')
plt.tight_layout()
plt.savefig('pos3.png',facecolor='w',edgecolor='w')

In [ ]:
plt.figure(dpi=400)
for i in ['PG','SG','SF','PF','C']:#np.unique(pose_short):
    plt.scatter(res[np.where(pose_short==i),0],res[np.where(pose_short==i),1],s=4,label=i)
    #for i,name in enumerate(pva.keys()):
    #    plt.text(res[i,0],res[i,1],name.split()[1],ha='center',va='center',size=3)
plt.legend(scatterpoints=4,markerscale=3)
plt.xticks([],[])
plt.yticks([],[])
plt.title('NBA Positions based on per-possession statistics (via UMAP)')
plt.tight_layout()
plt.savefig('pos2.png',facecolor='w',edgecolor='w')

In [ ]:
clusterer.cluster_centers_

In [ ]:
team['advanced'].iloc[:,5:-10].iloc[:,[0,4,5,9]]

In [ ]:
len(pose_short),t.shape